In [9]:
from keras import metrics
from model import neural_voice_judgment_model

model = neural_voice_judgment_model()
model.build((None,128,128,1))
model.summary()
model.compile(optimizer="adam",loss="mean_squared_error",metrics=metrics.TopKCategoricalAccuracy(k=5))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 124, 124, 8)       208       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 62, 62, 8)        0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 58, 58, 16)        3216      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 29, 29, 16)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 25, 25, 100)       40100     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 12, 12, 100)     

In [11]:
import numpy as np
from sample_yomikomi import sample_all_data
from keras.callbacks import EarlyStopping
import idx2numpy
import tensorflow as tf
from os.path import join
from twenty_times_more_data import twenty_times_more_data

input_output_label_data = idx2numpy.convert_from_file(join("..","dataset","train_label.idx"))
input_output_image_data = idx2numpy.convert_from_file(join("..","dataset","train_image.idx"))
label_dataset = tf.data.Dataset.from_tensor_slices(input_output_label_data)
image_dataset = tf.data.Dataset.from_tensor_slices(input_output_image_data)
lump_dataset = tf.data.Dataset.zip((image_dataset,label_dataset))
callback = EarlyStopping(patience=10,monitor="top_k_categorical_accuracy")
flat_map_dataset = lump_dataset.flat_map(twenty_times_more_data)
fit_data = model.fit(flat_map_dataset,epochs=500,steps_per_epoch=45,callbacks=[callback])

Epoch 1/500
45/45 [==============================] - 19s 410ms/step - loss: 0.1063 - top_k_categorical_accuracy: 0.9556
Epoch 2/500
45/45 [==============================] - 18s 409ms/step - loss: 0.1212 - top_k_categorical_accuracy: 1.0000
Epoch 3/500
45/45 [==============================] - 18s 410ms/step - loss: 0.1212 - top_k_categorical_accuracy: 1.0000
Epoch 4/500
45/45 [==============================] - 19s 412ms/step - loss: 0.1061 - top_k_categorical_accuracy: 1.0000
Epoch 5/500
45/45 [==============================] - 19s 412ms/step - loss: 0.1136 - top_k_categorical_accuracy: 1.0000
Epoch 6/500
45/45 [==============================] - 18s 410ms/step - loss: 0.1086 - top_k_categorical_accuracy: 1.0000
Epoch 7/500
45/45 [==============================] - 19s 411ms/step - loss: 0.1061 - top_k_categorical_accuracy: 1.0000
Epoch 8/500
45/45 [==============================] - 18s 410ms/step - loss: 0.0859 - top_k_categorical_accuracy: 1.0000
Epoch 9/500
45/45 [=====================

In [12]:
from keras.models import Model

test_label_data = idx2numpy.convert_from_file(join("..","dataset","test_label.idx"))
test_image_data = idx2numpy.convert_from_file(join("..","dataset","test_image.idx"))

In [13]:
model.evaluate(x=test_image_data,y=test_label_data)

32/32 [==============================] - 3s 91ms/step - loss: 0.1087 - top_k_categorical_accuracy: 1.0000


[0.1087045669555664, 1.0]

In [15]:
model.save(join("trained_model","second.tf"))

INFO:tensorflow:Assets written to: trained_model\second_tf\assets
